In [134]:
# TODO: Fix use of arrays in the dictionaries to just values

import pandas as pd
import numpy as np
import requests
from requests import get
from bs4 import BeautifulSoup
import os.path
import re
from datetime import datetime
import time
import concurrent.futures
import urllib.request
import ast
from stem import Signal
from stem.control import Controller
from os import listdir
from os.path import isfile, join

In [135]:
def get_tor_session():
    session = requests.session()
    # Tor uses the 9050 port as the default socks port
    session.proxies = {'http':  'socks5://127.0.0.1:9050',
                       'https': 'socks5://127.0.0.1:9050'}
    return session

def renew_connection():
    with Controller.from_port(port = 9051) as controller:
        controller.authenticate(password="password")
        controller.signal(Signal.NEWNYM)

In [136]:
if os.path.exists('./filtered_movies.pkl'):
    movies = pd.read_pickle("./filtered_movies.pkl")
else:
    title_basics = pd.read_csv("data/title.basics.tsv", sep='\t')
    movies = title_basics[title_basics.titleType == 'movie']
    movies.to_pickle("./filtered_movies.pkl")
movies.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
145,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,\N,20,"Documentary,News,Sport"
332,tt0000335,movie,Soldiers of the Cross,Soldiers of the Cross,0,1900,\N,\N,"Biography,Drama"
499,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N
571,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Biography,Crime,Drama"


In [137]:
len(movies)

534354

In [138]:
def get_data(session, url):
    scraped_data = {
    "tconst": [],
    "stars": [],
    "oscarWins": [],
    "nominations": [],
    "wins": [],
    "releaseDate": [],
    "releaseCountry": [],
    "plotKeywords": [],
    "budget": [],
    "worldwideGross": [],
    "metascore": [],
    "musicProducer": []
    }
    
    info=[]
    
    try:
        r = session.get(url)
        r.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)
    page_body = r.text
    soup = BeautifulSoup(page_body, 'html.parser')
    
    #tconst
    tconst=url.rsplit('/', 1)[-1]
    scraped_data['tconst'].append(tconst)
    
    
    with open("./ScrapedPages/tor/{0}.html".format(tconst), "w") as text_file:
        text_file.write(page_body)
        
    # Stars
    stars = []
    stars_h4 = soup.find('h4', string='Stars:')
    if stars_h4 is not None:
        star_atags_parent = stars_h4.parent
        if star_atags_parent is not None:
            star_atags = star_atags_parent.find_all('a')
            if star_atags is not None:
                for atag in star_atags:
                    if atag['href'].startswith('/name/'):
                        stars.append(atag['href'].split('/')[2])
    else:
        stars_h4 = soup.find('h4', string='Star:')
        if stars_h4 is not None:
            star_atags_parent = stars_h4.parent
            if star_atags_parent is not None:
                star_atags = star_atags_parent.find_all('a')
                if star_atags is not None:
                    for atag in star_atags:
                        if atag['href'].startswith('/name/'):
                            stars.append(atag['href'].split('/')[2])
    scraped_data['stars'].append(stars)
    
    
    # Metascore
    metascore = None
    metascore_list = soup.select('.metacriticScore span:first-child')
    if len(metascore_list) > 0:
        metascore = metascore_list[0].string
        
    scraped_data['metascore'].append(metascore if metascore is None else str(metascore))
    
    
    #awards
    awrds_lines = soup.find_all(class_="awards-blurb")
    oscars = 0
    wins = 0
    nominations = 0
    for line in awrds_lines:
        
        if line.findChild() is not None:
            prepped_str = re.sub(' +', ' ', line.findChild().text.replace("\n", " ").strip())
            res = re.search('(W|w)on (\d+) (O|o)scars.?', prepped_str)
            if res is not None:
                oscars = int(res.group(2))
            
        else:
            prepped_str = re.sub(' +', ' ', line.text.replace("\n", "").strip())
            
            res = re.search('(\d+) wins', prepped_str)
            if res is not None:
                wins = int(res.group(1))
            
            
            res = re.search('(\d+) nominations', prepped_str)
            if res is not None:
                nominations = int(res.group(1))
    scraped_data['oscarWins'].append(oscars)
    scraped_data['wins'].append(wins)
    scraped_data['nominations'].append(nominations)
    
    
    # Release date
    release_date_h4 = soup.find('h4', string='Release Date:')
    release_date = None
    release_country = None
    if release_date_h4 is not None:
        release_date_raw_text = release_date_h4.parent.findAll(text=True, recursive=False)
        release_date_prepped = re.sub(' +', ' ', ''.join(release_date_raw_text).replace("\n", "").strip())
        date_str_match = re.search(r'\d{1,2} \w+ \d{4}', release_date_prepped)
        if date_str_match is not None:
            release_date = datetime.strptime(date_str_match.group(), '%d %B %Y').date()
        release_country_match = re.search(r'\(([a-zA-Z ]{2,})\)', release_date_prepped)
        if release_country_match is not None and len(release_country_match.groups()) > 0:
            release_country = release_country_match.group(1)
        
    scraped_data['releaseDate'].append(release_date)
    scraped_data['releaseCountry'].append(release_country)
    
    
    # Budget
    budget_h4 = soup.find('h4', string='Budget:')
    budget = None
    if budget_h4 is not None:
        budget_raw_text = budget_h4.parent.findAll(text=True, recursive=False)
        budget = re.sub(' +', ' ', ''.join(budget_raw_text).replace("\n", "").strip())
        
    scraped_data['budget'].append(budget)
    
    
    # worldwide gross
    gross_h4 = soup.find('h4', string='Cumulative Worldwide Gross:')
    gross = None
    if gross_h4 is not None:
        gross_h4_text = gross_h4.parent.findAll(text=True, recursive=False)
        gross = re.sub(' +', ' ', ''.join(gross_h4_text).replace("\n", "").strip())
    
    scraped_data['worldwideGross'].append(gross)
    
    
    # Plot keywords
    keywords_verification_threshold = 2 # Consider only words atleast 2 people considered relavent
    keywords_url = url + "/keywords"
    r = get(keywords_url)
    page_body = r.text
    soup = BeautifulSoup(page_body, 'html.parser')
    keywords = []
    plot_keywords_items = soup.find_all(class_="soda sodavote")
    if plot_keywords_items is not None:
        for plot_keywords_item in plot_keywords_items:
            validity_text = plot_keywords_item.find(class_='interesting-count-text').a.text.strip()
            validity_text_match = re.search(r'(\d+) of', validity_text)
            if validity_text_match is not None and len(validity_text_match.groups()) > 0:
                if int(validity_text_match.group(1)) >= keywords_verification_threshold:
                    keywords.append(plot_keywords_item.find(class_='sodatext').a.text.strip())
    
    scraped_data['plotKeywords'].append(keywords)
    
    
    # Music producer
    fullcredits_url = url + "/fullcredits"
    r = get(fullcredits_url)
    page_body = r.text
    soup = BeautifulSoup(page_body, 'html.parser')
    
    music_producer = None
    
    full_credits_container = soup.find(id='fullcredits_content', class_='header')
    if full_credits_container is not None:
        full_credits = full_credits_container.find_all(recursive=False)
        if full_credits is not None:
            for idx, item in enumerate(full_credits, start=0):
                if 'Music by' in item.text:
                    producer_atag = full_credits[idx + 1].find('a')
                    if producer_atag is not None:
                        producer_href = producer_atag['href']
                        if producer_href is not None:
                            music_producer = producer_href.split('/')[2]
                            break
    
    scraped_data['musicProducer'].append(music_producer)
    return scraped_data

In [ ]:
# Set the start index and end index. Ip is renewed after each chunk of urls of 'step_size'. 
# After every chunk, result is written to a pickle. So if you need to stop the execution in the middle, note the
# index range of the last successfully written pickle file (from the printed logs) and use the remaining range
# for start and end index to resume from where you stopped.
postfix = 'tor'
s_idx = 0
e_idx = 133588
step_size = 4000
# [0:133588]

base_url = 'https://www.imdb.com/title/'
urls=[]
for index, row in movies[s_idx:e_idx].iterrows():
    urls.append(base_url + row['tconst'])

used_ips = []

with concurrent.futures.ThreadPoolExecutor(max_workers=150) as executor:
    session = get_tor_session()
    chunk_indeces = np.arange(s_idx, e_idx, step_size)
    
    for idx, start in enumerate(chunk_indeces, start=1):
        start_t = time.time()
        finals = {
            "tconst": [],
            "stars": [],
            "oscarWins": [],
            "nominations": [],
            "wins": [],
            "releaseDate": [],
            "releaseCountry": [],
            "plotKeywords": [],
            "budget": [],
            "worldwideGross": [],
            "metascore": [],
            "musicProducer": []
        }
        renew_connection()
        session = get_tor_session()
        new_ip = ast.literal_eval(session.get("http://httpbin.org/ip").text)["origin"].split(",")[0]
        while new_ip in used_ips:
            print("Renewed IP {0} already used. Waiting 5s to renew...".format(new_ip))
            time.sleep(5)
            renew_connection()
            session = get_tor_session()
            new_ip = ast.literal_eval(session.get("http://httpbin.org/ip").text)["origin"].split(",")[0]
        used_ips.append(new_ip)
        
        url_chunk = urls[start:start + step_size]
        
        future_to_url = {executor.submit(get_data, session, url): url for url in url_chunk}
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
                for k, v in data.items():
                    finals[k].append(v[0])
            except Exception as exc:
                print('%r generated an exception: %s' % (url, exc))
        
        concurrent.futures.wait(list(future_to_url.keys()), timeout=None, return_when=concurrent.futures.ALL_COMPLETED)
        end_t = time.time()
        print("chunk {0} of {1} completed with IP {2}. Took {3} seconds.".format(idx, len(chunk_indeces), new_ip, end_t - start_t))   
        movies_df = pd.DataFrame(data=finals)
        pickle_path = "./pickles/tor/{0}_{1}-{2}_{3}.pkl".format(idx, start, start + step_size, postfix)
        movies_df.to_pickle(pickle_path)
        print("Wrote pickle {0} with {1} rows".format(pickle_path, len(movies_df)))

chunk 1 of 34 completed with IP 31.220.29.166. Took 970.7494752407074 seconds.
Wrote pickle ./pickles/tor/1_0-4000_tor.pkl with 4000 rows


In [133]:
# Combine the created pickles into one Dataframe

dir_path = "./pickles/tor"
# Ignore any sytem files starting with . or folders if any
pickles = [f for f in listdir(dir_path) if isfile(join(dir_path, f)) and not f.startswith('.')]
dfs = list(map(lambda x: pd.read_pickle("{0}/{1}".format(dir_path,x)), pickles))
combined_df = pd.concat(dfs)
combined_df.reset_index(drop=True, inplace=True)
combined_df.to_pickle("./pickles/complete_{0}.pkl".format(e_idx))
with pd.option_context('display.max_rows', 50000, 'display.max_columns', 20):
    display(combined_df)


,tconst,stars,oscarWins,nominations,wins,releaseDate,releaseCountry,plotKeywords,budget,worldwideGross,metascore,musicProducer
0,tt0001440,"[nm0064953, nm0085066, nm1434907]",0,0,0,1910-08-29,Denmark,[],None,None,None,None
1,tt0001113,[],0,0,0,None,None,[],None,None,None,None
2,tt0001628,[nm0848232],0,0,0,None,None,[],None,None,None,None
3,tt0001602,"[nm0136241, nm0216941, nm0239263]",0,0,0,None,None,[],None,None,None,None
4,tt0001555,[],0,0,0,1911-10-07,Portugal,[],None,None,None,None
5,tt0001115,"[nm0064953, nm0085066, nm0169878]",0,0,0,1910-10-27,Denmark,[],None,None,None,None
6,tt0001366,[],0,0,0,None,None,[],None,None,None,None
7,tt0001592,"[nm1458590, nm0478475, nm1532213]",0,0,0,1911-08-12,Denmark,[],None,None,None,None
8,tt0001530,"[nm0738202, nm0198972, nm0519315]",0,0,0,1911-02-09,Australia,[],None,None,None,None
9,tt0001422,[nm0001908],0,0,0,None,None,[],None,None,None,None


In [144]:
# test_ctr = {
#     "tconst": [],
#     "stars": [],
#     "oscarWins": [],
#     "nominations": [],
#     "wins": [],
#     "releaseDate": [],
#     "releaseCountry": [],
#     "plotKeywords": [],
#     "budget": [],
#     "worldwideGross": [],
#     "metascore": [],
#     "musicProducer": []
# }

# def tempfn(x):
#     for index, value in x.items():
#         test_ctr[index].append(value[0]) 

# test_df = part1.apply(tempfn, axis=1)

In [ ]:
# renew_connection()
# session = get_tor_session()
# print(session.get("http://httpbin.org/ip").text)